In [5]:
def f(a,b):
    if len(a) !=0 and len(b) !=0:
        return (a+b)
    else:
        return 0

In [7]:
f([],[])

0

In [1]:
from __future__ import unicode_literals
import pandas as pd

In [2]:
pd.options.display.max_colwidth = -1
pd.options.display.max_columns = None

In [3]:
filepath = '~/Documents/Avito/'
df = pd.read_csv('ItemInfo_train5.csv',encoding='utf-8'
                 ,usecols=['description_x_clean','description_y_clean']
                 ,converters={'description_x_clean': lambda x: x.strip(u'[]').split(', ')
                 ,'description_y_clean': lambda x: x.strip('[]').split(', ')}
                ,nrows=10)
                             
#usecols=['description_clean'],

In [7]:
df.head()

The slowest run took 6.11 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 53 µs per loop


In [8]:
%timeit df['description_x_clean'][3]

The slowest run took 39.43 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 10.6 µs per loop


In [5]:
df['desc'] = df[['description_x_clean', 'description_y_clean']].values.tolist()

In [27]:
df.drop(['description_x_clean','description_y_clean'], axis=1, inplace=True)

In [28]:
df.head()

,description_similarity,desc
0,1.000000,"[[продам, камаз, 6520, 20, тонн], [продам, камаз, 6520, 20, тонн]]"
1,0.698324,"[[тюнинге], [тюнинге, возможен, торг]]"
2,0.902371,"[[телефон, хорошем, состоянии, трещин, сколов, менялся, аккумуляторпоэтому, заряд, держит, остальное, родное, целом, работает, отлично, комплекте, кабель, обмен, интересен, лежит, дела, продажа], [отличном, состоянии, комплекте, зарядник, кабель, обмена]]"
3,1.000000,"[[отличный, подарок, новый, китайской, apple, новый, упаковке, коробку, вместе, вскроем, экран, 5, дюймов, 1080p, sharpjdi, ogs, возможно, использование, перчатках, цвет, корпуса, белый, материалы, корпуса, пластик, металл, стекло, процессор, qualcomm, snapdragon, 801, 8x74ac, 25ггц, видео, adreno, 330, 578, мгц, озу, 3гб, lpddr3, 933мгц, память, 16гб, emmc, 50, камера, задняя, 13мп, sony, imx214, f18, 6, линз, камера, передняя, 8мп, sony, imx219, f18, 5, линз, wifi, 80211, abgnac, 245ггц, bluetooth, 40, навигация, gpsglonass, сенсоры, икпорт, датчик, освещенности, барометр, компас, датчик, силы, тяжести, симкарта, microsim, аккумулятор, 3080, мач, быстрая, зарядка, 9v12a, 5v2a, дополнительно, поддержка, smart, cover, case, ширина, 685, мм, высота, 1392, мм, толщина, ...], [отличный, подарок, новый, китайской, apple, новый, упаковке, коробку, вместе, вскроем, экран, 5, дюймов, 1080p, sharpjdi, ogs, возможно, использование, перчатках, цвет, корпуса, белый, материалы, корпуса, пластик, металл, стекло, процессор, qualcomm, snapdragon, 801, 8x74ac, 25ггц, видео, adreno, 330, 578, мгц, озу, 3гб, lpddr3, 933мгц, память, 16гб, emmc, 50, камера, задняя, 13мп, sony, imx214, f18, 6, линз, камера, передняя, 8мп, sony, imx219, f18, 5, линз, wifi, 80211, abgnac, 245ггц, bluetooth, 40, навигация, gpsglonass, сенсоры, икпорт, датчик, освещенности, барометр, компас, датчик, силы, тяжести, симкарта, microsim, аккумулятор, 3080, мач, быстрая, зарядка, 9v12a, 5v2a, дополнительно, поддержка, smart, cover, case, ширина, 685, мм, высота, 1392, мм, толщина, ...]]"
4,0.983680,"[[лыжные, ботинки, хорошем, состоянии, 34, размер], [ботинки, 34, размер, хорошем, состоянии]]"


### Word2Vec 
- compute similarity between 2 documents (`description`)

In [8]:
from gensim import models, similarities

Using TensorFlow backend.


In [ ]:
# model_new.wv.similarity('продам','размер')

In [ ]:
model_doc = models.doc2vec.Doc2Vec.load('description_doc')

In [9]:
model_word = models.word2vec.Word2Vec.load('description_2vec.model')

MemoryError: 

In [9]:
def remove_unknown_words(row):
    sent=[]
    for word in row:
        if word in set(model_word.wv.index2word):
            sent.append(word)
    return sent

In [4]:
def doc_sim(row):
    try:
        print(row[0])
        return  model_word.n_similarity(row[0],row[1])
        
    except:
        return model_word.n_similarity(remove_unknown_words(row[0]),
                                       remove_unknown_words(row[1]))

In [18]:
import multiprocessing as mp
from multiprocessing.dummy import Pool as ThreadPool
from contextlib import closing

In [ ]:
pool = ThreadPool(4)
%timeit df['description_similarity'] = pool.map(doc_sim,df['desc'])

In [ ]:
p = mp.Pool(3)
results = p.imap(doc_sim,df['desc'],10)

In [ ]:
results

In [ ]:
with closing(mp.Pool(3)) as p:
#     lst =[]
#     lst.append(p.imap(doc_sim,df,10))
    df['description_similarity'] = p.map(doc_sim,df)
    p.terminate()

In [6]:
df['description_similarity'] = df['desc'].apply(doc_sim)                                                                           

[u'\u043f\u0440\u043e\u0434\u0430\u043c', u'\u043a\u0430\u043c\u0430\u0437', u'6520', u'20', u'\u0442\u043e\u043d\u043d']


NameError: global name 'model_word' is not defined

In [17]:
df.description_similarity.isnull().sum() 

0

In [ ]:
lst_df = pd.DataFrame(lst,columns=['description_similarity'])

In [30]:
df.head()

,description_similarity,desc
0,1.000000,"[[продам, камаз, 6520, 20, тонн], [продам, камаз, 6520, 20, тонн]]"
1,0.698324,"[[тюнинге], [тюнинге, возможен, торг]]"
2,0.902371,"[[телефон, хорошем, состоянии, трещин, сколов, менялся, аккумуляторпоэтому, заряд, держит, остальное, родное, целом, работает, отлично, комплекте, кабель, обмен, интересен, лежит, дела, продажа], [отличном, состоянии, комплекте, зарядник, кабель, обмена]]"
3,1.000000,"[[отличный, подарок, новый, китайской, apple, новый, упаковке, коробку, вместе, вскроем, экран, 5, дюймов, 1080p, sharpjdi, ogs, возможно, использование, перчатках, цвет, корпуса, белый, материалы, корпуса, пластик, металл, стекло, процессор, qualcomm, snapdragon, 801, 8x74ac, 25ггц, видео, adreno, 330, 578, мгц, озу, 3гб, lpddr3, 933мгц, память, 16гб, emmc, 50, камера, задняя, 13мп, sony, imx214, f18, 6, линз, камера, передняя, 8мп, sony, imx219, f18, 5, линз, wifi, 80211, abgnac, 245ггц, bluetooth, 40, навигация, gpsglonass, сенсоры, икпорт, датчик, освещенности, барометр, компас, датчик, силы, тяжести, симкарта, microsim, аккумулятор, 3080, мач, быстрая, зарядка, 9v12a, 5v2a, дополнительно, поддержка, smart, cover, case, ширина, 685, мм, высота, 1392, мм, толщина, ...], [отличный, подарок, новый, китайской, apple, новый, упаковке, коробку, вместе, вскроем, экран, 5, дюймов, 1080p, sharpjdi, ogs, возможно, использование, перчатках, цвет, корпуса, белый, материалы, корпуса, пластик, металл, стекло, процессор, qualcomm, snapdragon, 801, 8x74ac, 25ггц, видео, adreno, 330, 578, мгц, озу, 3гб, lpddr3, 933мгц, память, 16гб, emmc, 50, камера, задняя, 13мп, sony, imx214, f18, 6, линз, камера, передняя, 8мп, sony, imx219, f18, 5, линз, wifi, 80211, abgnac, 245ггц, bluetooth, 40, навигация, gpsglonass, сенсоры, икпорт, датчик, освещенности, барометр, компас, датчик, силы, тяжести, симкарта, microsim, аккумулятор, 3080, мач, быстрая, зарядка, 9v12a, 5v2a, дополнительно, поддержка, smart, cover, case, ширина, 685, мм, высота, 1392, мм, толщина, ...]]"
4,0.983680,"[[лыжные, ботинки, хорошем, состоянии, 34, размер], [ботинки, 34, размер, хорошем, состоянии]]"
